In [1]:
# dataiku , change to basic data load

import karmahutils as kut
import pandas as pd
import numpy as np

loaded numpyutils ,version : moonshade library ,build: v1.18


# load data
## dataiku

In [2]:
data=kut.load_dataset("importedData")
data["sex"]=data["Personal status and sex"].apply(lambda x:int(x.split(':')[0]=='male'))
data.sample(n=4)

2021-09-28 10:52:00.173073 loading importedData
df importedData loaded: 1000 in 0:00:00.135844


,Age in years,Cost Matrix(Risk),Credit amount,Credit history,Duration in month,Housing,Installment rate in percentage of disposable income,Job,Number of existing credits at this bank,Number of people being liable to provide maintenance for,Other debtors / guarantors,Other installment plans,Personal status and sex,Present employment since,Present residence since,Property,Purpose,Savings account/bonds,Status of existing checking account,Telephone,foreign worker,sex
151,38,Good Risk,1924,existing credits paid back duly till now,10,own,1,skilled employee / official,1,1,none,none,male:single,1<= < 4 years,4,savings agreement/life insurance,radio/television,<100 DM,no checking account,yes,no,1
967,54,Good Risk,3568,existing credits paid back duly till now,15,rent,4,management/ highly qualified employee,1,1,none,bank,female:divorced/separated/married,>=7 years,2,car or other,radio/television,<100 DM,no checking account,yes,yes,0
476,24,Good Risk,2569,existing credits paid back duly till now,39,own,4,skilled employee / official,1,1,none,none,male:single,1<= < 4 years,4,car or other,car (used),500 <= < 1000 DM,no checking account,none,yes,1
236,23,Bad Risk,14555,existing credits paid back duly till now,6,own,1,unemployed/ unskilled - non-resident,1,1,none,none,male:single,unemployed,2,savings agreement/life insurance,car (new),no savings account,0 <= <200 DM,yes,yes,1


## classique load

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/ekimetrics/ethical-ai-toolkit/main/data/german-risk-scoring.csv")
data.head(2)

In [3]:
transformation={
    "sex":{0:1,1:0} 
    
    
}

# utils

In [4]:
def filter_on_values(df, value_list, column_name):
    original_index = df.index.name
    if not pd.isnull(original_index):
        df.reset_index(inplace=True)
    out = df.merge(pd.DataFrame(value_list, columns=[column_name]), on=[column_name])
    if not pd.isnull(original_index):
        df.set_index(original_index, inplace=True)
        out.set_index(original_index, inplace=True)
    return out

In [5]:
len(filter_on_values(df=data,value_list=[1],column_name='sex')),len(filter_on_values(df=data,value_list=[0],column_name='sex'))

(690, 310)

In [6]:
def counterfactualize_data(data,transformation):
    for feature in transformation.keys():
        target_column=data[feature]
        target_values=transformation[feature].keys()
        non_affected=[X for X in target_column.unique() if not X in target_values]
        countered=[filter_on_values(df=data,column_name='sex',value_list=non_affected)]
        for value in target_values :
            target_subset=filter_on_values(df=data,column_name=feature,value_list=[value])
            # we act on sub dataframes , an alternative (faster ?) would be to act on series and remerge(tbt)
            print(len(target_subset))
            replacement={value:transformation[feature][value]}
            print(replacement)
            target_subset[feature]=target_subset[feature].replace(replacement)
            countered.append(target_subset)
        
        return pd.concat(countered,ignore_index=True)
counterfactualized=counterfactualize_data(
    data=data,
    transformation=transformation
) 
print(len(counterfactualized))
kut.display_message('transformation results')
print('original')
print(data.sex.value_counts())
print('counterfactualized')
print(counterfactualized.sex.value_counts())

310
{0: 1}
690
{1: 0}
1000
**************************
* transformation results *
**************************
original
1    690
0    310
Name: sex, dtype: int64
counterfactualized
0    690
1    310
Name: sex, dtype: int64
